# **EXPERIMENT DATA PREPROCESSING**

In [1]:
import pandas as pd
import numpy as np

# I. Import data

In [ ]:
data = pd.read_parquet('HFEA_full.parquet')

In [ ]:
data.shape

(1376454, 96)

## 1. Filtering

In [ ]:
HFEA_1999_2016 = data.copy()

# Keep only the years 2006 to 2016 
HFEA_2006_2016 = HFEA_1999_2016[HFEA_1999_2016['YEAR_OF_TREATMENT'].isin([2006, 2007, 2008, 2009, 2010, 2011, 2012, 2013, 2014, 2015, 2016])]
print('The number of cycles between 2006 and 2016 is '+str(HFEA_2006_2016.shape[0]))

original_data_rows = HFEA_2006_2016.shape[0]

# Remove surrogate cases
before_remove_rows = HFEA_2006_2016.shape[0]
HFEA_2006_2016 = HFEA_2006_2016[(HFEA_2006_2016['PATIENT_ACTING_AS_SURROGATE'] == 0)]
removed_rows = before_remove_rows - HFEA_2006_2016.shape[0]
print(f'Removed {removed_rows} surrogate cases. This is {100*removed_rows/original_data_rows:.2f}% of the original data.')

# Remove donated eggs and donated embryos
before_remove_rows = HFEA_2006_2016.shape[0]
HFEA_2006_2016 = HFEA_2006_2016[(HFEA_2006_2016['EGG_SOURCE'] == 'Patient')]
removed_rows = before_remove_rows - HFEA_2006_2016.shape[0]
print(f'Removed {removed_rows} cases with donated eggs and embryos. This is {100*removed_rows/original_data_rows:.2f}% of the original data.')

# Filter data to only keep patients currently undergoing IVF
before_remove_rows = HFEA_2006_2016.shape[0]
HFEA_2006_2016 = HFEA_2006_2016[(HFEA_2006_2016['MAIN_REASON_FOR_PRODUCING_EMBROYS_STORING_EGGS'] == 'Treatment Now ')]
removed_rows = before_remove_rows - HFEA_2006_2016.shape[0]
print(f'Removed {removed_rows} patients not undergoing treatment in this cycle. This is {100*removed_rows/original_data_rows:.2f}% of the original data.')

# Remove the ICSI:Unknown, IVF:Unknown, ICSI / BLASTOCYST, FER, ICSI / AH categories
    # Group the IVF:IVF with IVF
    # Group the ICSI:ICSI, ICSI:IVF, and IVF:ICSI with ICSI
before_remove_rows = HFEA_2006_2016.shape[0]
replace_values = {'ICSI:ICSI': 'ICSI', 'IVF:IVF': 'IVF', 'IVF:ICSI': 'ICSI', 'ICSI:IVF': 'ICSI'}
HFEA_2006_2016['SPECIFIC_TREATMENT_TYPE'] = HFEA_2006_2016['SPECIFIC_TREATMENT_TYPE'].replace(replace_values)
values_to_keep = ['IVF', 'ICSI']
HFEA_2006_2016 = HFEA_2006_2016[HFEA_2006_2016['SPECIFIC_TREATMENT_TYPE'].isin(values_to_keep)]
removed_rows = before_remove_rows - HFEA_2006_2016.shape[0]
print(f'Removed {removed_rows} patients not undergoing IVF or ICSI. This is {100*removed_rows/original_data_rows:.2f}% of the original data.')

# Remove rows with neither Fresh nor Frozen, or both at once or problematic
before_remove_rows = HFEA_2006_2016.shape[0]
mask = ((HFEA_2006_2016['FRESH_CYCLE'] == 0) & (HFEA_2006_2016['FROZEN_CYCLE'] == 0))|((HFEA_2006_2016['FRESH_CYCLE'] == 1) & (HFEA_2006_2016['FROZEN_CYCLE'] == 1))
HFEA_2006_2016 = HFEA_2006_2016.loc[~mask]

HFEA_2006_2016 = HFEA_2006_2016[HFEA_2006_2016['EMBRYOS_STORED_FOR_USE_BY_PATIENT']!='> 50']

HFEA_2006_2016['EMBRYOS_STORED_FOR_USE_BY_PATIENT'] = HFEA_2006_2016['EMBRYOS_STORED_FOR_USE_BY_PATIENT'].astype(float).astype('int64')
HFEA_2006_2016 = HFEA_2006_2016.loc[~((HFEA_2006_2016['EMBRYOS_STORED_FOR_USE_BY_PATIENT'] > 1) & (HFEA_2006_2016['EMBRYOS_TRANSFERED'] == 0))]
removed_rows = before_remove_rows - HFEA_2006_2016.shape[0]
print(f'Removed {removed_rows} cases with neither Fresh nor Frozen, or both at once. This is {100*removed_rows/original_data_rows:.2f}% of the original data.')

# Clean necessary for further processing
HFEA_2006_2016['FRESH_EGGS_COLLECTED'] =  HFEA_2006_2016['FRESH_EGGS_COLLECTED'].replace('> 50', 51)
HFEA_2006_2016['TOTAL_EMBRYOS_CREATED'] =  HFEA_2006_2016['TOTAL_EMBRYOS_CREATED'].replace('> 50', 51)
HFEA_2006_2016[['EMBRYOS_TRANSFERED', 'EMBRYOS_STORED_FOR_USE_BY_PATIENT', 'TOTAL_EMBRYOS_CREATED', 'FRESH_EGGS_COLLECTED']] = HFEA_2006_2016[['EMBRYOS_TRANSFERED', 'EMBRYOS_STORED_FOR_USE_BY_PATIENT', 'TOTAL_EMBRYOS_CREATED', 'FRESH_EGGS_COLLECTED']].astype(float).astype('int64')

# Remove patients who had more embryos created than eggs collected(annormality)
before_remove_rows = HFEA_2006_2016.shape[0]
HFEA_2006_2016 = HFEA_2006_2016[~((HFEA_2006_2016['TOTAL_EMBRYOS_CREATED']) > HFEA_2006_2016['FRESH_EGGS_COLLECTED'])]
removed_rows = before_remove_rows - HFEA_2006_2016.shape[0]
print(f'Removed {removed_rows} patients with more embryos created than eggs collected. This is {100*removed_rows/original_data_rows:.2f}% of the original data.')

# Check for duplicated rows
if HFEA_2006_2016.duplicated().any() == True:
    print('The dataset contains duplicated rows.')
else:
    print('The dataset does not contain any duplicated rows.')

# Drop irrelevant features
cols_before = HFEA_2006_2016.shape[1]
columns_to_drop = ['ROW_NUMBER', 'FROZEN_CYCLE', 'PATIENT_ACTING_AS_SURROGATE', 'MAIN_REASON_FOR_PRODUCING_EMBROYS_STORING_EGGS',
'DONATED_EMBRYO', 'TYPE_OF_TREATMENT__IVF_OR_DI', 'EGG_SOURCE', 'EGG_DONOR_AGE_AT_REGISTRATION', 'NUMBER_OF_LIVE_BIRTHS', 'EARLY_OUTCOME', 
'NUMBER_OF_FOETAL_SACS_WITH_FETAL_PULSATION','EGGS_THAWED', 'TOTAL_EMBRYOS_THAWED',
'FRESH_EGGS_STORED', 'TOTAL_EGGS_MIXED', 'EGGS_MIXED_WITH_PARTNER_SPERM', 'EGGS_MIXED_WITH_DONOR_SPERM',
'EGGS_MICROINJECTED', 'EMBRYOS_FROM_EGGS_MICROINJECTED', 'EMBRYOS_FROM_EGGS_MICROINJECTED_STORED_FOR_USE_BY_PATIENT',
'DATE_OF_EGG_COLLECTION', 'DATE_OF_EGG_THAWING', 'DATE_OF_EGG_MIXING', 'DATE_OF_EMBRYO_THAWING', 'DATE_OF_EMBRYO_TRANSFER', 
'ELECTIVE_SINGLE_EMBRYO_TRANSFER', 'EMBRYOS_TRANSFERED_FROM_EGGS_MICROINJECTED', 'EMBRYOS_TRANSFERED_FROM_EGGS_MICROINJECTED',
'EMBRYOS_STORED_FOR_USE_BY_PATIENT']

HFEA_2006_2016 = HFEA_2006_2016.drop(columns=columns_to_drop)
HFEA_2006_2016 = HFEA_2006_2016.drop(columns=HFEA_2006_2016.filter(regex='HEART').columns)
removed_cols = cols_before - HFEA_2006_2016.shape[1]
print(f'Removed {removed_cols} columns.')

print(f'The number of FRESH+FROZEN, exploitable cycles from 2016 is {HFEA_2006_2016.shape[0]}')
print(f'This represents {100*HFEA_2006_2016.shape[0]/original_data_rows:.2f}% of the original data.')

The number of cycles between 2006 and 2016 is 712157
Removed 50327 surrogate cases. This is 7.07% of the original data.
Removed 27941 cases with donated eggs and embryos. This is 3.92% of the original data.
Removed 37812 patients not undergoing treatment in this cycle. This is 5.31% of the original data.
Removed 86743 patients not undergoing IVF or ICSI. This is 12.18% of the original data.
Removed 13619 cases with neither Fresh nor Frozen, or both at once. This is 1.91% of the original data.
Removed 659 patients with more embryos created than eggs collected. This is 0.09% of the original data.
The dataset does not contain any duplicated rows.
Removed 51 columns.
The number of FRESH+FROZEN, exploitable cycles from 2016 is 495056
This represents 69.52% of the original data.


## 2. Cleaning

In [ ]:
# Remove TYPE_OF_OVULATION_INDUCTION because this feature is constant except for 1 value or a few missing
# Remove the STARTED_TRYING feature because it is pratically empty for every cycle
# Remove PGD_TREATMENT and PGS_TREATMENT because their information is redundant
HFEA_2006_2016_cleaned = HFEA_2006_2016.drop(columns=['TYPE_OF_OVULATION_INDUCTION', 
'DATE_PATIENT_STARTED_TRYING_TO_BECOME_PREGNANT_OR_DATE_OF_LAST_PREGNANCY', 'PGD_TREATMENT', 'PGS_TREATMENT'])

# The existing feature SPERM_FROM will be used to capture this information.
HFEA_2006_2016_cleaned = HFEA_2006_2016_cleaned.drop(columns=['SPERM_DONOR_AGE_AT_REGISTRATION'])

# For PGS:
    # Fill miising values with zeros because this is what they are
    # Feature is extremely sparse and could maybe just be removed direcly (same applies for PGD)
HFEA_2006_2016_cleaned['PGS'] = HFEA_2006_2016_cleaned['PGS'].fillna(0)

HFEA_2006_2016_cleaned['SCREENING'] = HFEA_2006_2016_cleaned['PGS'] + HFEA_2006_2016_cleaned['PGD']
HFEA_2006_2016_cleaned['SCREENING'] = HFEA_2006_2016_cleaned['SCREENING'].clip(upper=1)

HFEA_2006_2016_cleaned = HFEA_2006_2016_cleaned.drop(['PGS', 'PGD'], axis=1)

# Fill nans in LIVE_BIRTH_OCCURRENCE with zeros because this is what they are
HFEA_2006_2016_cleaned['LIVE_BIRTH_OCCURRENCE'] = HFEA_2006_2016_cleaned['LIVE_BIRTH_OCCURRENCE'].fillna(0)

# Check for constant features
constant_features = HFEA_2006_2016_cleaned.columns[HFEA_2006_2016_cleaned.nunique() <= 1]
if len(constant_features) != 0:
    print("There are constant features. The following features are constant:")
    for feature in constant_features:
        print(feature)
else:
    print('There are no constant features.')

# Drop the constant features with respect to 2016 data
HFEA_2006_2016_cleaned = HFEA_2006_2016_cleaned.drop(['CAUSE_OF_INFERTILITY__CERVICAL_FACTORS', 'CAUSE_OF_INFERTILITY__FEMALE_FACTORS', 
'CAUSE_OF_INFERTILITY___PARTNER_SPERM_IMMUNOLOGICAL_FACTORS'], axis=1)

There are constant features. The following features are constant:
CAUSE_OF_INFERTILITY__FEMALE_FACTORS


## 3. Encodings / Dummifications

In [ ]:
# Some variables are counts but are capped, i.e. >=5. These need to be transformed. 
# We will chose the next greatest value in the series as an approximationion.
HFEA_2006_2016_cleaned[['TOTAL_NUMBER_OF_PREVIOUS_CYCLES_BOTH_IVF_AND_DI',
'TOTAL_NUMBER_OF_PREVIOUS_TREATMENTS_BOTH_IVF_AND_DI_AT_CLINIC',
'TOTAL_NUMBER_OF_PREVIOUS_IVF_CYCLES',
'TOTAL_NUMBER_OF_PREVIOUS_DI_CYCLES']] = HFEA_2006_2016_cleaned[['TOTAL_NUMBER_OF_PREVIOUS_CYCLES_BOTH_IVF_AND_DI',
'TOTAL_NUMBER_OF_PREVIOUS_TREATMENTS_BOTH_IVF_AND_DI_AT_CLINIC',
'TOTAL_NUMBER_OF_PREVIOUS_IVF_CYCLES',
'TOTAL_NUMBER_OF_PREVIOUS_DI_CYCLES']].replace('>=5', 6)

# Dummify the SPECIFIC_TREATMENT_TYPE variable
HFEA_2006_2016_cleaned = pd.get_dummies(HFEA_2006_2016_cleaned, columns=['SPECIFIC_TREATMENT_TYPE'], prefix='SPECIFIC_TREATMENT_TYPE', drop_first=True)

# The variables PATIENT_AGE_AT_TREATMENT and SPERM_DONOR_AGE_AT_REGISTRATION need to be encoded and coserve their oridnal nature. 
# We simply replace categories with integers corresponding to their respective position
replace_values = {'18 - 34': 1, '35-37': 2, '38-39': 3, '40-42': 4, '43-44':5, '45-50':6}
HFEA_2006_2016_cleaned['PATIENT_AGE_AT_TREATMENT'] = HFEA_2006_2016_cleaned['PATIENT_AGE_AT_TREATMENT'].replace(replace_values)

replace_values = {'Partner': 0, 'Donor': 1, 'not assigned':0, 'Partner & Donor':0}
HFEA_2006_2016_cleaned['SPERM_FROM'] = HFEA_2006_2016_cleaned['SPERM_FROM'].replace(replace_values)

# Only done here because these features are not to be kept later anyway
replace_values = {'>=5': 6}
HFEA_2006_2016_cleaned[['TOTAL_NUMBER_OF_PREVIOUS_PREGNANCIES_BOTH_IVF_AND_DI', 
'TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_IVF_OR_DI',
'TOTAL_NUMBER_OF_IVF_PREGNANCIES']] = HFEA_2006_2016_cleaned[['TOTAL_NUMBER_OF_PREVIOUS_PREGNANCIES_BOTH_IVF_AND_DI', 
'TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_IVF_OR_DI',
'TOTAL_NUMBER_OF_IVF_PREGNANCIES']].replace(replace_values)

# Now that all variables have been encoded, every variable can be set to the int64 type since they are either count, ordinal, or binary.
HFEA_2006_2016_cleaned = HFEA_2006_2016_cleaned.astype('int64')

## Remove Correlated Features (with respect to 2016 data)

In [ ]:
HFEA_2006_2016_uncorr = HFEA_2006_2016_cleaned.copy()

HFEA_2006_2016_uncorr['TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_IVF'] = np.where(HFEA_2006_2016_uncorr['TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_IVF'] > 0, 1, 0)
HFEA_2006_2016_uncorr['TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_DI'] = np.where(HFEA_2006_2016_uncorr['TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_DI'] > 0, 1, 0)
HFEA_2006_2016_uncorr['PREVIOUS_LIVE_BIRTH_IVF_OR_DI'] = HFEA_2006_2016_uncorr['TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_IVF'] + HFEA_2006_2016_uncorr['TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_DI']
HFEA_2006_2016_uncorr['PREVIOUS_LIVE_BIRTH_IVF_OR_DI'] = np.where(HFEA_2006_2016_uncorr['PREVIOUS_LIVE_BIRTH_IVF_OR_DI'] > 0, 1, 0)

HFEA_2006_2016_uncorr.drop(columns=['TYPE_OF_INFERTILITY__FEMALE_PRIMARY', 'TYPE_OF_INFERTILITY__MALE_PRIMARY', 'TYPE_OF_INFERTILITY__FEMALE_SECONDARY',
'TYPE_OF_INFERTILITY__MALE_SECONDARY', 'TOTAL_NUMBER_OF_PREVIOUS_TREATMENTS_BOTH_IVF_AND_DI_AT_CLINIC',
'TOTAL_NUMBER_OF_IVF_PREGNANCIES', 'TOTAL_NUMBER_OF_DI_PREGNANCIES', 'TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_IVF', 
'TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_DI', 'TOTAL_NUMBER_OF_PREVIOUS_IVF_CYCLES', 'TOTAL_NUMBER_OF_PREVIOUS_DI_CYCLES', 
'TOTAL_NUMBER_OF_PREVIOUS_PREGNANCIES_BOTH_IVF_AND_DI', 'TOTAL_NUMBER_OF_LIVE_BIRTHS__CONCEIVED_THROUGH_IVF_OR_DI',
'TYPE_OF_INFERTILITY_COUPLE_PRIMARY', 'TYPE_OF_INFERTILITY_COUPLE_SECONDARY', 'CAUSE_OF_INFERTILITY__PARTNER_SPERM_CONCENTRATION',
'CAUSE_OF_INFERTILITY___PARTNER_SPERM_MORPHOLOGY', 'CAUSES_OF_INFERTILITY__PARTNER_SPERM_MOTILITY'] , axis=1, inplace=True)

## 2016 FRESH + FROZEN DATASET

In [ ]:
HFEA_2016_FRESH_FROZEN = HFEA_2006_2016_uncorr.drop(columns=['EMBRYOS_TRANSFERED'], axis=1)
HFEA_2016_FRESH_FROZEN = HFEA_2016_FRESH_FROZEN[HFEA_2016_FRESH_FROZEN['YEAR_OF_TREATMENT']==2016]
HFEA_2016_FRESH_FROZEN = HFEA_2016_FRESH_FROZEN.drop(columns=['YEAR_OF_TREATMENT'], axis=1)
HFEA_2016_FRESH_FROZEN.to_parquet('FRESH_FROZEN_2016.parquet')

## Build a dataset of fresh rounds per year

In [ ]:
# Remove Frozen cycles
before_remove_rows = HFEA_2006_2016_uncorr.shape[0]
HFEA_2006_2016_fresh = HFEA_2006_2016_uncorr[HFEA_2006_2016_uncorr['FRESH_CYCLE'] == 1]
removed_rows = before_remove_rows - HFEA_2006_2016_fresh.shape[0]
print(f'Removed {removed_rows} Frozen cycles. This is {100*removed_rows/original_data_rows:.2f}% of the original data.')
print('There are '+str(HFEA_2006_2016_fresh.shape[0])+' FRESH cycles in the dataset.')

# Remove patients who had a live birth despite no eggs collected (annormality)
before_remove_rows = HFEA_2006_2016_fresh.shape[0]
HFEA_2006_2016_fresh = HFEA_2006_2016_fresh[~((HFEA_2006_2016_fresh['FRESH_EGGS_COLLECTED'] == 0) & (HFEA_2006_2016_fresh['LIVE_BIRTH_OCCURRENCE'] == 1))]
removed_rows = before_remove_rows - HFEA_2006_2016_fresh.shape[0]
print(f'Removed {removed_rows} patients with live birth despite no eggs collected. This is {100*removed_rows/original_data_rows:.2f}% of the original data.')

# Remove 2016
HFEA_2006_2015_fresh = HFEA_2006_2016_fresh[HFEA_2006_2016_fresh['YEAR_OF_TREATMENT']!=2016]
HFEA_2006_2015_fresh= HFEA_2006_2015_fresh.drop(columns=['EMBRYOS_TRANSFERED'], axis=1)

Removed 51839 Frozen cycles. This is 7.28% of the original data.
There are 443217 FRESH cycles in the dataset.
Removed 23 patients with live birth despite no eggs collected. This is 0.00% of the original data.


In [ ]:
# Function to save a dataframe per year and drop the YEAR_OF_TREATMENT_VARIABLE
def split_and_save_by_year(df):

    # Get the unique years in the 'YEAR' column
    unique_years = df['YEAR_OF_TREATMENT'].unique()
    
    # Iterate over the unique years
    for year in unique_years:
        # Filter the DataFrame to include only the current year
        df_year = df[df['YEAR_OF_TREATMENT'] == year]
        df_year = df_year.drop(columns=['YEAR_OF_TREATMENT'], axis=1)
        
        # Define the filename using a coherent naming scheme
        filename = f"HFEA_{year}.parquet"
        
        # Save the filtered DataFrame as a Parquet file
        df_year.to_parquet(filename)
        
        # Print a message indicating progress
        print(f"Saved data for year {year} to {filename}")


In [ ]:
split_and_save_by_year(HFEA_2006_2015_fresh)

Saved data for year 2015 to HFEA_2015.parquet
Saved data for year 2010 to HFEA_2010.parquet
Saved data for year 2011 to HFEA_2011.parquet
Saved data for year 2012 to HFEA_2012.parquet
Saved data for year 2013 to HFEA_2013.parquet
Saved data for year 2014 to HFEA_2014.parquet
Saved data for year 2009 to HFEA_2009.parquet
Saved data for year 2006 to HFEA_2006.parquet
Saved data for year 2008 to HFEA_2008.parquet
Saved data for year 2007 to HFEA_2007.parquet
